<a href="https://colab.research.google.com/github/cjsdudwls1/ai_tutor-test.ver-/blob/main/ai%ED%8A%9C%ED%84%B0(%ED%85%8C%EC%8A%A4%ED%8A%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 필요한 패키지 설치 (버전 명시)
!pip install faiss-cpu langchain-text-splitters python-docx unstructured langchain_community llama-index gradio docx2txt -qqq
!pip install --upgrade "google-genai>=1.7.0" "langchain-google-genai>=2.1.3" -qqq


# **1단계 : 문서 로드**

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("/content/drive/MyDrive/(난이도표시본) 샘플문제 워드로 다시.docx")  # 문서 로더 초기화

docs = loader.load()  # 문서 로딩

print(len(docs))


1


### llm이 못알아 들을 수 있으니 <br>**정규 표현식을 사용하여 '하 N.', '중 N.', '상 N.'을 '난이도 하 N.', '난이도 중 N.', '난이도 상 N.'으로 변환**

In [ ]:
import re

def transform_difficulty_markers(docs):
    for i, doc in enumerate(docs):
        modified_content = re.sub(r'(하|중|상)( \d+\.)', r'난이도 \1\2', doc.page_content)
        docs[i].page_content = modified_content
    return docs

# 변환된 문서 사용
docs = transform_difficulty_markers(docs)
print(docs[0].page_content)

샘플문제 (정답 및 해설 포함)

난이도 하 1. 다음 중 간(liver)에서 수행하지 않는 주요 기능은 무엇인가?

A. 알부민(albumin) 합성

B. 담즙(bile) 생성

C. 혈당 조절(glucose regulation)

D. 인슐린(insulin) 분비

정답: D. 인슐린(insulin) 분비
 해설: 인슐린은 췌장(이자)에서 분비되며, 간은 알부민 합성, 담즙 생성, 혈당 조절을 담당합니다.

난이도 중 2. 근수축(muscle contraction)에 직접적으로 관여하는 단백질 복합체로 옳은 것은?

A. 미오신(myosin)과 트로포닌(troponin)

B. 트로포닌(troponin)과 트로포마이오신(tropomyosin)

C. 액틴(actin)과 콜라겐(collagen)

D. 엘라스틴(elastin)과 미오신(myosin)

정답: B. 트로포닌(troponin)과 트로포마이오신(tropomyosin)
 해설: 트로포닌과 트로포마이오신은 근수축 시 칼슘과 결합하여 액틴-미오신 상호작용을 조절합니다.

난이도 하 3. 다음 중 혈액-뇌 장벽(blood-brain barrier)을 통과하기 가장 쉬운 물질은?

A. 포도당(glucose)

B. 산소(oxygen)

C. 단백질(protein)

D. 나트륨 이온(sodium ion)

정답: B. 산소(oxygen)
 해설: 산소는 지용성 분자로, 혈액-뇌 장벽을 가장 쉽게 통과할 수 있습니다.

난이도 중 4. 신장에서 여과(filtration)가 일어나는 주요 구조물은?

A. 보우만주머니(Bowman's capsule)

B. 근위세뇨관(proximal convoluted tubule)

C. 헨레고리(loop of Henle)

D. 집합관(collecting duct)

정답: A. 보우만주머니(Bowman's capsule)
 해설: 신장에서 혈액의 여과는 사구체와 보우만주머니에서 일어납니다.

난이도 중 5. 다음 중 자율신경계(autonomic nervous syst

# **2단계: 텍스트 분할**

In [ ]:
import re
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

# 2. 청킹을 위한 정규식 패턴 정의
pattern = r'(난이도 하 \d+\.|난이도 중 \d+\.|난이도 상 \d+\.)'

# 3. 커스텀 분할 함수
def question_splitter(text, pattern):
    splits = re.split(pattern, text)
    chunks = []
    for i in range(1, len(splits), 2):
        if i+1 < len(splits):
            chunk = splits[i] + splits[i+1]
            chunks.append(chunk.strip())
    return chunks

# 4. 문서 내용 추출 및 분할
full_text = docs[0].page_content  # 로드된 문서의 전체 텍스트
question_chunks = question_splitter(full_text, pattern)

# 5. 결과 확인
for idx, chunk in enumerate(question_chunks[:3]):  # 상위 3개 샘플 출력
    print(f"=== 문제 {idx+1} ===")
    print(chunk)
    print("\n")


=== 문제 1 ===
난이도 하 1. 다음 중 간(liver)에서 수행하지 않는 주요 기능은 무엇인가?

A. 알부민(albumin) 합성

B. 담즙(bile) 생성

C. 혈당 조절(glucose regulation)

D. 인슐린(insulin) 분비

정답: D. 인슐린(insulin) 분비
 해설: 인슐린은 췌장(이자)에서 분비되며, 간은 알부민 합성, 담즙 생성, 혈당 조절을 담당합니다.


=== 문제 2 ===
난이도 중 2. 근수축(muscle contraction)에 직접적으로 관여하는 단백질 복합체로 옳은 것은?

A. 미오신(myosin)과 트로포닌(troponin)

B. 트로포닌(troponin)과 트로포마이오신(tropomyosin)

C. 액틴(actin)과 콜라겐(collagen)

D. 엘라스틴(elastin)과 미오신(myosin)

정답: B. 트로포닌(troponin)과 트로포마이오신(tropomyosin)
 해설: 트로포닌과 트로포마이오신은 근수축 시 칼슘과 결합하여 액틴-미오신 상호작용을 조절합니다.


=== 문제 3 ===
난이도 하 3. 다음 중 혈액-뇌 장벽(blood-brain barrier)을 통과하기 가장 쉬운 물질은?

A. 포도당(glucose)

B. 산소(oxygen)

C. 단백질(protein)

D. 나트륨 이온(sodium ion)

정답: B. 산소(oxygen)
 해설: 산소는 지용성 분자로, 혈액-뇌 장벽을 가장 쉽게 통과할 수 있습니다.




# **3단계: 임베딩**

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

# 1. 모델과 토크나이저 로드
model_name = 'BM-K/KoSimCSE-roberta'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. 임베딩 추출 함수 수정 (문장 수준 임베딩)
def get_embeddings(sentences):
    # 입력: 문장 리스트 (예: ["문장1", "문장2", ...])
    inputs = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        max_length=512,  # 최대 길이 제한
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**inputs)

    # [CLS] 토큰 임베딩 추출 (첫 번째 토큰)
    # outputs.last_hidden_state.shape = (batch_size, seq_len, hidden_dim)
    cls_embeddings = outputs.last_hidden_state[:, 0, :]

    # 정규화 (선택 사항)
    cls_embeddings = torch.nn.functional.normalize(cls_embeddings, p=2, dim=1)

    return cls_embeddings

# 3. 예시: 청킹된 문제 리스트를 임베딩
embeddings = get_embeddings(question_chunks)  # (num_chunks, 768) 형태 텐서

# 4. 임베딩 결과 확인
print(embeddings.shape)  # → torch.Size([문항개수, 768])


torch.Size([10, 768])


# **4단계: FAISS 벡터 스토어 생성**

In [ ]:
import numpy as np
import faiss
import pickle
import os

# 저장 경로 설정
save_dir = '/content/drive/MyDrive/tutor_store'

# 1. 임베딩을 NumPy 배열로 변환 (KoSimCSE 출력이 PyTorch 텐서라 가정)
embeddings_np = embeddings.numpy().astype('float32')  # (num_chunks, 768)

# 2. FAISS 인덱스 생성 (L2 거리 기준)
dimension = embeddings_np.shape[1]  # 768
faiss_index = faiss.IndexFlatL2(dimension)

# 3. 임베딩을 인덱스에 추가
faiss_index.add(embeddings_np)

# 4. 벡터스토어와 청크 매핑 저장
faiss_path = os.path.join(save_dir, "medical_qa_index.faiss")
chunk_path = os.path.join(save_dir, "chunk_mapping.pkl")

# FAISS 인덱스 저장
faiss.write_index(faiss_index, faiss_path)
print(f"FAISS 인덱스가 {faiss_path}에 저장되었습니다.")

# 청크 리스트 저장 (검색 시 텍스트 복원용)
with open(chunk_path, 'wb') as f:
    pickle.dump(question_chunks, f)
print(f"청크 매핑이 {chunk_path}에 저장되었습니다.")

FAISS 인덱스가 /content/drive/MyDrive/tutor_store/medical_qa_index.faiss에 저장되었습니다.
청크 매핑이 /content/drive/MyDrive/tutor_store/chunk_mapping.pkl에 저장되었습니다.


# 5단계: tutor 시스템 구현 (gemini-1.5-pro)

## **라이브러리 임포트 및 기본 설정**

In [ ]:
import os
import faiss
import numpy as np
import pickle
import re
import gradio as gr
from google.colab import drive, userdata
from typing import List, Dict, Optional, Any
from langchain_community.vectorstores.faiss import FAISS as LangchainFAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import FakeEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import TextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore


# 드라이브 마운트 및 API 키 설정
drive.mount('/content/drive')
os.environ['GOOGLE_API_KEY'] = userdata.get('Gemini_API_KEY')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **벡터 스토어 경로 설정**

In [ ]:
store_dir = '/content/drive/MyDrive/tutor_store'
faiss_path = os.path.join(store_dir, "medical_qa_index.faiss")
chunk_path = os.path.join(store_dir, "chunk_mapping.pkl")

## **세션 상태 관리 클래스**




1. SessionState 클래스는 사용자의 난이도(current_level), 정답/오답 개수, 현재 문제·정답 보관, 이미 본 문제 ID 집합을 관리합니다.

2. total_attempts, success_rate 프로퍼티로 시도 횟수와 성공률을 계산합니다.

3. update_stats() 메서드로 정답 여부에 따라 카운트를 갱신하고, 일정 기준 이상일 때 난이도를 상향/하향 조정합니다.

In [ ]:
class SessionState:
    def __init__(self):
        self.current_level = "난이도 하"
        self.correct_count = 0
        self.incorrect_count = 0
        self.current_question = None
        self.current_answer = None
        self.question_ids = set()

    @property
    def total_attempts(self):
        return self.correct_count + self.incorrect_count

    @property
    def success_rate(self):
        return (self.correct_count / self.total_attempts) if self.total_attempts else 0

    def update_stats(self, is_correct: bool):
        if is_correct:
            self.correct_count += 1
            if self.current_level == "난이도 하" and self.success_rate >= 0.7 and self.total_attempts >= 3:
                self.current_level = "난이도 중"
            elif self.current_level == "난이도 중" and self.success_rate >= 0.7 and self.total_attempts >= 3:
                self.current_level = "난이도 상"
        else:
            self.incorrect_count += 1
            if self.current_level == "난이도 상" and self.success_rate < 0.5:
                self.current_level = "난이도 중"
            elif self.current_level == "난이도 중" and self.success_rate < 0.3:
                self.current_level = "난이도 하"


##  **MedicalQuestionSplitter (커스텀 텍스트 분할기)**
1. TextSplitter를 상속하여, “난이도 하 1.”처럼 각 문제 제목 패턴별로 텍스트를 분할합니다.

2. split_text()는 정규표현식으로 짝수 인덱스의 제목과 문제 본문을 합쳐서 청크(문제 단위) 리스트를 반환합니다.

In [ ]:
class MedicalQuestionSplitter(TextSplitter):
    def __init__(self):
        super().__init__()
        self.pattern = r'(난이도 하 \d+\.|난이도 중 \d+\.|난이도 상 \d+\.)'

    def split_text(self, text: str) -> List[str]:
        splits = re.split(self.pattern, text)
        chunks = []
        for i in range(1, len(splits), 2):
            if i+1 < len(splits):
                chunk = splits[i] + splits[i+1]
                chunks.append(chunk.strip())
        return chunks


## **CustomFakeEmbeddings (커스텀 임베딩)**

1.
FAISS 인덱스 로드 시, 실제 임베딩 대신 모두 0벡터를 반환하는 FakeEmbeddings를 활용합니다.

2. embed_documents(), embed_query() 모두 고정 길이 0벡터를 생성하여 반환합니다.



In [ ]:
class CustomFakeEmbeddings(FakeEmbeddings):
    def __init__(self, size: int = 1024):
        super().__init__(size=size)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [np.zeros(self.size) for _ in texts]

    def embed_query(self, text: str) -> List[float]:
        return np.zeros(self.size)

## **벡터스토어 및 문서 로드 함수**
1. FAISS 인덱스 파일을 읽어오고, pickle로 저장된 문제 청크 리스트를 불러옵니다.

2. 각 청크를 Document 객체로 래핑하며, 레벨 정보는 get_level_from_text()로 메타데이터에 저장합니다.

3. 앞서 정의한 CustomFakeEmbeddings를 사용해 LangChain FAISS 벡터스토어를 초기화한 후, docstore에 문서를 매핑합니다.

In [ ]:
def load_vectorstore():
    print("FAISS 인덱스와 청크 매핑을 로드하는 중...")
    index = faiss.read_index(faiss_path)
    with open(chunk_path, 'rb') as f:
        question_chunks = pickle.load(f)
    documents = [Document(page_content=chunk, metadata={"level": get_level_from_text(chunk)}) for chunk in question_chunks]
    embeddings = CustomFakeEmbeddings(size=index.d)
    vectorstore = LangchainFAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    vectorstore.docstore._dict = {str(i): doc for i, doc in enumerate(documents)}
    return vectorstore, documents


## **유틸리티 함수들**
1. get_level_from_text(): 텍스트 맨 앞의 “난이도” 정보를 추출합니다.

2. process_question(): 본문에서 문제, 정답, 해설을 분리하여 튜플로 반환합니다.

3. hide_answer_from_question(): 문제 텍스트에서 정답 부분을 제거해 학생에게만 질문만 보여줍니다.

In [ ]:
def get_level_from_text(text: str) -> str:
    if text.startswith("난이도 하"): return "난이도 하"
    if text.startswith("난이도 중"): return "난이도 중"
    if text.startswith("난이도 상"): return "난이도 상"
    match = re.match(r'(난이도\s*[하중상])', text)
    return match.group(1) if match else "난이도 하"

def process_question(text: str) -> tuple:
    if "정답:" not in text:
        return text, "", ""
    q_part = text.split("정답:")[0].strip()
    a_full = text.split("정답:")[1].split("\n")[0].strip()
    a_part = a_full.split('.')[0].strip() if '.' in a_full else a_full
    exp = text.split("해설:")[1].strip() if "해설:" in text else ""
    return q_part, a_part, exp

def hide_answer_from_question(text: str) -> str:
    return text.split("정답:")[0].strip() if "정답:" in text else text


## **RAG(검색연계생성) 체인 및 리트리버 설정**

In [ ]:
def setup_rag_chain():
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
    vectorstore, documents = load_vectorstore()
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    return llm, retriever, documents


## **사용자 맞춤 질문 선택기 생성**
1. 현재 세션의 난이도와 이미 본 문제 목록을 기준으로, 아직 보지 않은 문제를 필터링합니다.

2. 해당 난이도의 문제가 부족하면 다른 난이도로 변경하거나, 모두 소진 시 question_ids를 초기화해 재출제합니다.

3. 선택된 문서를 반환하는 CustomRetriever 클래스를 정의하여 LangChain 리트리버 인터페이스와 호환시킵니다.

In [ ]:
def create_question_selector(documents: List[Document], session: SessionState):
    def filter_by_level_and_seen(query: str):
        level = session.current_level
        filtered_docs = [
            doc for doc in documents
            if doc.metadata.get("level") == level and hash(doc.page_content) not in session.question_ids
        ]
        if not filtered_docs:
            for lvl in ["난이도 하", "난이도 중", "난이도 상"]:
                if lvl != level:
                    filtered_docs = [
                        doc for doc in documents
                        if doc.metadata.get("level") == lvl and hash(doc.page_content) not in session.question_ids
                    ]
                    if filtered_docs:
                        session.current_level = lvl
                        break
            if not filtered_docs:
                session.question_ids.clear()
                filtered_docs = [doc for doc in documents if doc.metadata.get("level") == session.current_level]
        if filtered_docs:
            selected = np.random.choice(filtered_docs)
            session.question_ids.add(hash(selected.page_content))
            return [selected]
        return []
    class CustomRetriever:
        def get_relevant_documents(self, query: str) -> List[Document]:
            return filter_by_level_and_seen(query)
    return CustomRetriever()


## **프롬프트 템플릿 정의**
1. 출제용과 피드백용 두 가지 ChatPrompt 템플릿을 정의합니다.

2. question_template은 문제만 학생에게 보여주고, feedback_template은 정답 판단 및 해설에 따라 LLM에게 피드백 생성 지침을 제공합니다.

In [ ]:
question_template = ChatPromptTemplate.from_template("""
다음은 작업치료사 국가고시 문제입니다:

{question}

이 문제를 학생에게 출제하되, 정답과 해설은 포함하지 마세요.
""")

feedback_template = ChatPromptTemplate.from_messages([
    ("system", """당신은 작업치료사 국가고시 전문가입니다.
     학생의 답변을 평가하고 유익한 피드백을 제공해주세요."""),
    ("human", """
    문제: {question}
    학생 답변: {user_answer}
    정답: {correct_answer}
    정답 여부: {is_correct}
    해설: {explanation}

    다음 지침에 따라 피드백을 제공해주세요:
    1. 학생의 이해도를 격려하세요
    2. 왜 해당 답변이 맞거나 틀렸는지 간략히 설명하세요
    3. 핵심 개념을 한 문장으로 요약하세요
    """)
])


## **피드백 체인 생성 함수**
1. 최초 호출 시에만 LLM과 feedback_template을 연결한 체인을 생성하고 캐시합니다.
2. 이후로는 생성된 체인을 재활용하여 효율을 높입니다.

In [ ]:
def get_feedback_chain():
    if not hasattr(get_feedback_chain, "chain"):
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
        get_feedback_chain.chain = (
            RunnablePassthrough()
            | feedback_template
            | llm
            | StrOutputParser()
        )
    return get_feedback_chain.chain


## **문제 선택 및 질문 준비 함수**
1. select_next_question(): question_selector를 통해 다음 문제 Document를 가져옵니다.

2. prepare_next_question(): 학습 진행 상황을 문자열로 조합하고, 다음 문제만 학생에게 보여줄 형태로 반환합니다.

In [ ]:
def select_next_question(session: SessionState, question_selector) -> Optional[Document]:
    docs = question_selector.get_relevant_documents("next_question")
    return docs[0] if docs else None

def prepare_next_question(session: SessionState, question_selector) -> str:
    next_doc = select_next_question(session, question_selector)
    if not next_doc:
        return "**모든 문제를 풀었습니다! 수고하셨습니다.**"
    q, a, exp = process_question(next_doc.page_content)
    session.current_question = next_doc.page_content
    session.current_answer = a
    q_only = hide_answer_from_question(q)
    info = (f"**학습 진행 상황**\n"
            f"- 현재 난이도: {session.current_level}\n"
            f"- 푼 문제: {session.total_attempts}문제\n"
            f"- 맞춘 문제: {session.correct_count}문제\n"
            f"- 틀린 문제: {session.incorrect_count}문제\n\n")
    return info + f"**다음 문제**\n{q_only}"


## **피드백 생성 함수**
1. 학생 답안을 받아 정답 여부 플래그와 함께 LLM 피드백 체인에 전달하여 설명을 생성합니다.

2. 예외 발생 시 기본 텍스트 피드백으로 대체하고, 오답일 경우 정답과 해설을 추가로 붙여줍니다.

In [ ]:
def generate_feedback(session: SessionState, user_answer: str, is_correct: bool) -> str:
    try:
        question, _, exp = process_question(session.current_question)
        feedback_input = {
            "question": question,
            "user_answer": user_answer,
            "correct_answer": session.current_answer,
            "is_correct": "정답입니다!" if is_correct else "오답입니다.",
            "explanation": exp if exp else "추가 해설 없음"
        }
        fb = get_feedback_chain().invoke(feedback_input)
    except Exception as e:
        print(f"피드백 생성 중 오류 발생: {e}")
        fb = ("정답입니다!" if is_correct else "오답입니다.") + f" 정답은 {session.current_answer}입니다."
    if not is_correct:
        fb += f"\n\n**정답**: {session.current_answer}"
    _, _, exp = process_question(session.current_question)
    if exp:
        fb += f"\n\n**해설**: {exp}"
    return fb


## **Gradio 채팅 인터페이스 로직**
사용자가 첫 메시지를 보낼 때는 인사와 첫 문제를 보여주고, 이후로는 사용자의 답안을 평가·피드백 생성·다음 문제 제시 과정을 수행합니다.

In [ ]:
def chat_interface(message, history, session_state, question_selector):
    if not message or not message.strip():
        return history, ""
    if not history:
        doc = select_next_question(session_state, question_selector)
        if not doc:
            return [{"role": "assistant", "content": "문제를 불러올 수 없습니다."}], ""
        q, a, _ = process_question(doc.page_content)
        session_state.current_question = doc.page_content
        session_state.current_answer = a
        return [
            {"role": "assistant", "content": "안녕하세요! 작업치료사 국가고시 문제 풀이를 시작합니다."},
            {"role": "assistant", "content": hide_answer_from_question(q)}
        ], ""
    new_hist = history.copy()
    user_ans = message.strip().upper()
    new_hist.append({"role": "user", "content": user_ans})
    correct = (user_ans == session_state.current_answer.upper())
    session_state.update_stats(correct)
    fb = generate_feedback(session_state, user_ans, correct)
    nxt_q = prepare_next_question(session_state, question_selector)
    new_hist.append({"role": "assistant", "content": fb + "\n\n" + nxt_q})
    return new_hist, ""


## **버튼 클릭 및 채팅 초기화 함수**
1. answer_click()는 버튼 클릭 시 답안을 처리하고 채팅 내역을 갱신합니다.

2. clear_chat()는 세션과 채팅 내역을 초기화하여 첫 문제부터 다시 시작합니다.

In [ ]:
def answer_click(answer, history, session_state, question_selector):
    new_history, _ = chat_interface(answer, history, session_state, question_selector)
    return new_history

def clear_chat(session_state, question_selector):
    new_sess = SessionState()
    first_doc = select_next_question(new_sess, question_selector)
    if not first_doc:
        return new_sess, [{"role": "assistant", "content": "문제를 불러올 수 없습니다."}]
    q, a, _ = process_question(first_doc.page_content)
    new_sess.current_question = first_doc.page_content
    new_sess.current_answer = a
    return new_sess, [
        {"role": "assistant", "content": "안녕하세요! 작업치료사 국가고시 문제 풀이를 시작합니다."},
        {"role": "assistant", "content": hide_answer_from_question(q)}
    ]


## **메인 애플리케이션 및 Gradio UI 구성**
1. RAG 체인·세션·질문 선택기를 준비한 후, Gradio Blocks로 전체 UI를 구성합니다.

2. 답안 버튼(A/B/C/D)과 초기화 버튼을 배치하고, 각 클릭 이벤트에 앞서 정의한 함수를 바인딩합니다.

3. 앱을 외부 공유 모드(share=True)로 실행합니다.

In [ ]:
def main():
    llm, retriever, documents = setup_rag_chain()
    session_state = SessionState()
    question_selector = create_question_selector(documents, session_state)
    with gr.Blocks(css="""
        .gradio-container {max-width: 900px; margin: auto;}
        .chat-history {height: 600px !important;}
        footer {display: none !important;}
    """) as demo:
        gr.Markdown("# 🎓 작업치료사 국가고시 문제 학습 챗봇")
        gr.Markdown("버튼을 눌러 답안을 선택하세요.")
        session_state_gr = gr.State(session_state)
        question_selector_gr = gr.State(question_selector)
        chatbot = gr.Chatbot(label="학습 대화", height=500, type="messages", render_markdown=True)
        with gr.Row():
            btn_A = gr.Button("A")
            btn_B = gr.Button("B")
            btn_C = gr.Button("C")
            btn_D = gr.Button("D")
            clear_btn = gr.Button("초기화")
        btn_A.click(
            lambda history, ss, qs: answer_click("A", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        btn_B.click(
            lambda history, ss, qs: answer_click("B", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        btn_C.click(
            lambda history, ss, qs: answer_click("C", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        btn_D.click(
            lambda history, ss, qs: answer_click("D", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        clear_btn.click(
            lambda ss, qs: clear_chat(ss, qs),
            inputs=[session_state_gr, question_selector_gr],
            outputs=[session_state_gr, chatbot]
        )
        demo.load(
            lambda ss, qs: clear_chat(ss, qs),
            inputs=[session_state_gr, question_selector_gr],
            outputs=[session_state_gr, chatbot]
        )
    demo.launch(share=True)

if __name__ == "__main__":
    main()


FAISS 인덱스와 청크 매핑을 로드하는 중...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c55bdcff5ccafc76c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 전체코드

In [ ]:
아래의 코드를 구간 별 순서대로 설명해줘. 너가 해준 설명과 구간화를 그대로 나의COLAP작업환경에 셀로 구분하여 올릴거야.
import os
import faiss
import numpy as np
import pickle
import re
import gradio as gr
from google.colab import drive, userdata
from typing import List, Dict, Optional, Any
from langchain_community.vectorstores.faiss import FAISS as LangchainFAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import FakeEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import TextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore

# 드라이브 마운트 및 API 키 설정
drive.mount('/content/drive')
os.environ['GOOGLE_API_KEY'] = userdata.get('Gemini_API_KEY')

# 벡터스토어 경로 설정
store_dir = '/content/drive/MyDrive/tutor_store'
faiss_path = os.path.join(store_dir, "medical_qa_index.faiss")
chunk_path = os.path.join(store_dir, "chunk_mapping.pkl")

# 세션 상태 저장 클래스
class SessionState:
    def __init__(self):
        self.current_level = "난이도 하"
        self.correct_count = 0
        self.incorrect_count = 0
        self.current_question = None
        self.current_answer = None
        self.question_ids = set()

    @property
    def total_attempts(self):
        return self.correct_count + self.incorrect_count

    @property
    def success_rate(self):
        return (self.correct_count / self.total_attempts) if self.total_attempts else 0

    def update_stats(self, is_correct: bool):
        if is_correct:
            self.correct_count += 1
            if self.current_level == "난이도 하" and self.success_rate >= 0.7 and self.total_attempts >= 3:
                self.current_level = "난이도 중"
            elif self.current_level == "난이도 중" and self.success_rate >= 0.7 and self.total_attempts >= 3:
                self.current_level = "난이도 상"
        else:
            self.incorrect_count += 1
            if self.current_level == "난이도 상" and self.success_rate < 0.5:
                self.current_level = "난이도 중"
            elif self.current_level == "난이도 중" and self.success_rate < 0.3:
                self.current_level = "난이도 하"

# 의학 문제를 처리하기 위한 커스텀 텍스트 분할기
class MedicalQuestionSplitter(TextSplitter):
    def __init__(self):
        super().__init__()
        self.pattern = r'(난이도 하 \d+\.|난이도 중 \d+\.|난이도 상 \d+\.)'

    def split_text(self, text: str) -> List[str]:
        splits = re.split(self.pattern, text)
        chunks = []
        for i in range(1, len(splits), 2):
            if i+1 < len(splits):
                chunk = splits[i] + splits[i+1]
                chunks.append(chunk.strip())
        return chunks

# 로드된 FAISS 인덱스와 호환되는 커스텀 FakeEmbeddings
class CustomFakeEmbeddings(FakeEmbeddings):
    def __init__(self, size: int = 1024):
        super().__init__(size=size)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [np.zeros(self.size) for _ in texts]

    def embed_query(self, text: str) -> List[float]:
        return np.zeros(self.size)

# 벡터스토어 및 문제 문서 로드
def load_vectorstore():
    print("FAISS 인덱스와 청크 매핑을 로드하는 중...")
    index = faiss.read_index(faiss_path)
    with open(chunk_path, 'rb') as f:
        question_chunks = pickle.load(f)
    documents = [Document(page_content=chunk, metadata={"level": get_level_from_text(chunk)}) for chunk in question_chunks]
    embeddings = CustomFakeEmbeddings(size=index.d)
    vectorstore = LangchainFAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    vectorstore.docstore._dict = {str(i): doc for i, doc in enumerate(documents)}
    return vectorstore, documents

# 유틸리티 함수들
def get_level_from_text(text: str) -> str:
    if text.startswith("난이도 하"): return "난이도 하"
    if text.startswith("난이도 중"): return "난이도 중"
    if text.startswith("난이도 상"): return "난이도 상"
    match = re.match(r'(난이도\s*[하중상])', text)
    return match.group(1) if match else "난이도 하"

def process_question(text: str) -> tuple:
    if "정답:" not in text:
        return text, "", ""
    q_part = text.split("정답:")[0].strip()
    a_full = text.split("정답:")[1].split("\n")[0].strip()
    a_part = a_full.split('.')[0].strip() if '.' in a_full else a_full
    exp = text.split("해설:")[1].strip() if "해설:" in text else ""
    return q_part, a_part, exp

def hide_answer_from_question(text: str) -> str:
    return text.split("정답:")[0].strip() if "정답:" in text else text

# LLM 및 RAG 설정
def setup_rag_chain():
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
    vectorstore, documents = load_vectorstore()
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    return llm, retriever, documents

# 문제 선택 리트리버 생성
def create_question_selector(documents: List[Document], session: SessionState):
    def filter_by_level_and_seen(query: str):
        level = session.current_level
        filtered_docs = [
            doc for doc in documents
            if doc.metadata.get("level") == level and hash(doc.page_content) not in session.question_ids
        ]
        if not filtered_docs:
            for lvl in ["난이도 하", "난이도 중", "난이도 상"]:
                if lvl != level:
                    filtered_docs = [
                        doc for doc in documents
                        if doc.metadata.get("level") == lvl and hash(doc.page_content) not in session.question_ids
                    ]
                    if filtered_docs:
                        session.current_level = lvl
                        break
            if not filtered_docs:
                session.question_ids.clear()
                filtered_docs = [doc for doc in documents if doc.metadata.get("level") == session.current_level]
        if filtered_docs:
            selected = np.random.choice(filtered_docs)
            session.question_ids.add(hash(selected.page_content))
            return [selected]
        return []
    class CustomRetriever:
        def get_relevant_documents(self, query: str) -> List[Document]:
            return filter_by_level_and_seen(query)
    return CustomRetriever()

# 프롬프트 템플릿 정의
question_template = ChatPromptTemplate.from_template("""
다음은 작업치료사 국가고시 문제입니다:

{question}

이 문제를 학생에게 출제하되, 정답과 해설은 포함하지 마세요.
""")

feedback_template = ChatPromptTemplate.from_messages([
    ("system", """당신은 작업치료사 국가고시 전문가입니다.
     학생의 답변을 평가하고 유익한 피드백을 제공해주세요."""),
    ("human", """
    문제: {question}
    학생 답변: {user_answer}
    정답: {correct_answer}
    정답 여부: {is_correct}
    해설: {explanation}

    다음 지침에 따라 피드백을 제공해주세요:
    1. 학생의 이해도를 격려하세요
    2. 왜 해당 답변이 맞거나 틀렸는지 간략히 설명하세요
    3. 핵심 개념을 한 문장으로 요약하세요
    """)
])

# 피드백 체인 생성 함수 (전역 객체로 캐싱)
def get_feedback_chain():
    if not hasattr(get_feedback_chain, "chain"):
        llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
        get_feedback_chain.chain = (
            RunnablePassthrough()
            | feedback_template
            | llm
            | StrOutputParser()
        )
    return get_feedback_chain.chain

# 메인 애플리케이션 함수
def select_next_question(session: SessionState, question_selector) -> Optional[Document]:
    docs = question_selector.get_relevant_documents("next_question")
    return docs[0] if docs else None

def prepare_next_question(session: SessionState, question_selector) -> str:
    next_doc = select_next_question(session, question_selector)
    if not next_doc:
        return "**모든 문제를 풀었습니다! 수고하셨습니다.**"
    q, a, exp = process_question(next_doc.page_content)
    session.current_question = next_doc.page_content
    session.current_answer = a
    q_only = hide_answer_from_question(q)
    info = (f"**학습 진행 상황**\n"
            f"- 현재 난이도: {session.current_level}\n"
            f"- 푼 문제: {session.total_attempts}문제\n"
            f"- 맞춘 문제: {session.correct_count}문제\n"
            f"- 틀린 문제: {session.incorrect_count}문제\n\n")
    return info + f"**다음 문제**\n{q_only}"

def generate_feedback(session: SessionState, user_answer: str, is_correct: bool) -> str:
    try:
        question, _, exp = process_question(session.current_question)
        feedback_input = {
            "question": question,
            "user_answer": user_answer,
            "correct_answer": session.current_answer,
            "is_correct": "정답입니다!" if is_correct else "오답입니다.",
            "explanation": exp if exp else "추가 해설 없음"
        }
        fb = get_feedback_chain().invoke(feedback_input)
    except Exception as e:
        print(f"피드백 생성 중 오류 발생: {e}")
        fb = ("정답입니다!" if is_correct else "오답입니다.") + f" 정답은 {session.current_answer}입니다."
    if not is_correct:
        fb += f"\n\n**정답**: {session.current_answer}"
    _, _, exp = process_question(session.current_question)
    if exp:
        fb += f"\n\n**해설**: {exp}"
    return fb

# Gradio UI 함수
def chat_interface(message, history, session_state, question_selector):
    if not message or not message.strip():
        return history, ""
    if not history:
        doc = select_next_question(session_state, question_selector)
        if not doc:
            return [{"role": "assistant", "content": "문제를 불러올 수 없습니다."}], ""
        q, a, _ = process_question(doc.page_content)
        session_state.current_question = doc.page_content
        session_state.current_answer = a
        return [
            {"role": "assistant", "content": "안녕하세요! 작업치료사 국가고시 문제 풀이를 시작합니다."},
            {"role": "assistant", "content": hide_answer_from_question(q)}
        ], ""
    new_hist = history.copy()
    user_ans = message.strip().upper()
    new_hist.append({"role": "user", "content": user_ans})
    correct = (user_ans == session_state.current_answer.upper())
    session_state.update_stats(correct)
    fb = generate_feedback(session_state, user_ans, correct)
    nxt_q = prepare_next_question(session_state, question_selector)
    new_hist.append({"role": "assistant", "content": fb + "\n\n" + nxt_q})
    return new_hist, ""

def answer_click(answer, history, session_state, question_selector):
    new_history, _ = chat_interface(answer, history, session_state, question_selector)
    return new_history

def clear_chat(session_state, question_selector):
    new_sess = SessionState()
    first_doc = select_next_question(new_sess, question_selector)
    if not first_doc:
        return new_sess, [{"role": "assistant", "content": "문제를 불러올 수 없습니다."}]
    q, a, _ = process_question(first_doc.page_content)
    new_sess.current_question = first_doc.page_content
    new_sess.current_answer = a
    return new_sess, [
        {"role": "assistant", "content": "안녕하세요! 작업치료사 국가고시 문제 풀이를 시작합니다."},
        {"role": "assistant", "content": hide_answer_from_question(q)}
    ]

# 메인 애플리케이션
def main():
    llm, retriever, documents = setup_rag_chain()
    session_state = SessionState()
    question_selector = create_question_selector(documents, session_state)
    with gr.Blocks(css="""
        .gradio-container {max-width: 900px; margin: auto;}
        .chat-history {height: 600px !important;}
        footer {display: none !important;}
    """) as demo:
        gr.Markdown("# 🎓 작업치료사 국가고시 문제 학습 챗봇")
        gr.Markdown("버튼을 눌러 답안을 선택하세요.")
        session_state_gr = gr.State(session_state)
        question_selector_gr = gr.State(question_selector)
        chatbot = gr.Chatbot(label="학습 대화", height=500, type="messages", render_markdown=True)
        with gr.Row():
            btn_A = gr.Button("A")
            btn_B = gr.Button("B")
            btn_C = gr.Button("C")
            btn_D = gr.Button("D")
            clear_btn = gr.Button("초기화")
        btn_A.click(
            lambda history, ss, qs: answer_click("A", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        btn_B.click(
            lambda history, ss, qs: answer_click("B", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        btn_C.click(
            lambda history, ss, qs: answer_click("C", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        btn_D.click(
            lambda history, ss, qs: answer_click("D", history, ss, qs),
            inputs=[chatbot, session_state_gr, question_selector_gr],
            outputs=[chatbot]
        )
        clear_btn.click(
            lambda ss, qs: clear_chat(ss, qs),
            inputs=[session_state_gr, question_selector_gr],
            outputs=[session_state_gr, chatbot]
        )
        demo.load(
            lambda ss, qs: clear_chat(ss, qs),
            inputs=[session_state_gr, question_selector_gr],
            outputs=[session_state_gr, chatbot]
        )
    demo.launch(share=True)

if __name__ == "__main__":
    main()
